### Create model risk drivers
- This notebook is designed to create a Long List of risk drivers for a mortgage model.
- We are utilizing input data from Fannie Mae and leveraging ChatGPT to assist in identifying and documenting risk drivers.
- The goal is to provide a structured JSON array with detailed explanations for each driver, categorized appropriately.

**The list has not been properly reviewed, use at you own risk**

In [1]:
import pandas as pd
import requests
from io import BytesIO

# Ensure no limitations in Jupyter Notebook output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### A. Get Fannie Mae data

In [2]:
# URL of the Excel file
ref = 'https://capitalmarkets.fanniemae.com/resources/file/credit-risk/xls/crt-file-layout-and-glossary.xlsx'

# Fetch the Excel file from the URL
response = requests.get(ref)

# Check if the request was successful
if response.status_code == 200:
    # Read the Excel file into a pandas DataFrame
    data = pd.read_excel(BytesIO(response.content))
else:
    print(f"Failed to fetch file. Status code: {response.status_code}")

data.head()

,Field Position,Field Name,Description,Enumerations,Date Bound Notes,Respective Disclosure Notes,CAS,CIRT,Single-Family (SF) Loan Performance,Type,Max Length
0,1.0,Reference Pool ID,A unique identifier for the reference pool.,NaN,NaN,NaN,√,√,NaN,ALPHA-NUMERIC,X(4)
1,2.0,Loan Identifier,A unique identifier for the mortgage loan.,NaN,NaN,The Loan Identifier does not correspond to other mortgage loan identifiers found in existing Fannie Mae disclosures.,√,√,√,ALPHA-NUMERIC,X(12)
2,3.0,Monthly Reporting Period,The month and year that pertains to the servicer’s cut-off period for mortgage loan information.,NaN,SF Loan Performance: Enhanced format with the October 2020 Release,NaN,√,√,√,DATE,MMYYYY
3,4.0,Channel,The origination channel used by the party that delivered the loan to the issuer.,R = Retail\nC = Correspondent\nB = Broker,NaN,NaN,√,√,√,ALPHA-NUMERIC,X(1)
4,5.0,Seller Name,The name of the entity that delivered the mortgage loan to Fannie Mae.,NaN,NaN,"CAS/CIRT: For sellers whose combined loans' contribution to the At Issuance UPB represents less than 1% of the total At Issuance reference pool UPB, the file will reflect ""Other"".",√,√,√,ALPHA-NUMERIC,X(50)


### B. Create prompt

#### Prompt

You are a credit risk modeller tasked with creating a Long List of risk drivers for a mortgage model. Please create a JSON array where each element is an object with the following keys:

1. **"Driver"**: The data field’s name, verbatim or slightly shortened if needed.  
2. **"Explanation"**: A concise description of why this field is relevant to loan underwriting, servicing, credit risk assessment, or performance monitoring. You should specifically connect each field to a potential impact on default, prepayment, or loss severity when possible.  
3. **"Category"**: A top-level grouping that classifies the field (e.g., `"Loan Identification"`, `"Loan Terms"`, `"Borrower & Credit Profile"`, `"Collateral & Equity"`, `"Loan Performance Tracking"`, `"Servicing & Administration"`, `"Foreclosure & Disposal"`, etc.).

Make sure the JSON output is valid and properly formatted with the following structure:

```json
{
  "Driver": "",
  "Explanation": "",
  "Category": ""
},
{
  "Driver": "",
  "Explanation": "",
  "Category": ""
}

### C. Get Long List from the OpenAI o1 Pro

In [3]:
long_list_dict = [
  {
    "Driver": "Prompt Payment Record",
    "Explanation": "Evaluates how quickly the borrower makes monthly payments; late or missed payments strongly correlate with higher default risk.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "On-Time Payment Streak",
    "Explanation": "Tracks consecutive months of timely payments. Longer streaks usually signal reliable repayment and lower credit risk.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Highest Missed Payment",
    "Explanation": "Records the largest unpaid or partially paid amount in any billing cycle. Significant missed amounts may point to severe financial stress.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Partial Payment Incidence",
    "Explanation": "Counts how often the borrower pays less than the required amount. Frequent short payments imply liquidity constraints or budgeting issues.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Late Fee Frequency",
    "Explanation": "Captures how often the borrower incurs late charges. Repeated late fees suggest habitual repayment difficulties.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Stable Payment Method",
    "Explanation": "Monitors whether the borrower changes payment methods often (e.g., switching from autopay to manual). Instability can hint at inconsistent financial planning.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Rolling Balance Pattern",
    "Explanation": "Observes whether the borrower continually carries an outstanding balance. Persistent rollovers may signal rising debt burden.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Reliance on Minimum Payments",
    "Explanation": "Checks how frequently the borrower only meets the minimum due. Chronic minimum-only payments are often a red flag for strained finances.",
    "Category": "Payment Behavior"
  },
  {
    "Driver": "Total Credit Usage Ratio",
    "Explanation": "Measures the ratio of all current credit balances to the combined credit limit. High overall utilization correlates with greater default probability.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Card-Level Usage Rate",
    "Explanation": "Focuses on the ratio of the balance to the limit for a specific card. High usage on a single card can precede delinquency.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Usage Volatility",
    "Explanation": "Examines fluctuations in credit utilization over time. Large swings may reflect income variability or sudden spending shocks.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Limit Overrun Count",
    "Explanation": "Counts instances when the borrower exceeds the credit limit. Going over limit is a major warning sign of liquidity trouble.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Credit Limit Upsurge Requests",
    "Explanation": "Tracks how often the borrower asks for an increased credit limit. Frequent requests can imply urgent cash needs or credit-seeking behavior.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Balance Shift Activity",
    "Explanation": "Identifies how often balances are transferred between accounts. A high volume of balance transfers may indicate debt reshuffling or liquidity strains.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Cash Draw Frequency",
    "Explanation": "Measures how often the borrower takes cash advances. Frequent withdrawals point to urgent cash needs and higher risk.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Aggregate Cash Advance",
    "Explanation": "Summarizes the total amount withdrawn via cash advances. Large aggregated amounts may reflect severe financial stress.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Purchase Velocity",
    "Explanation": "Tracks how often the account is used for purchases. Sudden spikes or drops can signal changing financial habits or instability.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Transaction Size Average",
    "Explanation": "Looks at the typical amount spent per transaction. A surge might indicate overspending, while a consistent decline could suggest caution or income issues.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Spending Category Distribution",
    "Explanation": "Analyzes where the borrower spends (e.g., essentials vs. luxury). A switch toward necessity spending may reflect tightening finances.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Global Transaction Count",
    "Explanation": "Tracks the frequency of international purchases. Changes in foreign spending may reflect personal or economic shifts that can affect risk.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Declined Transactions Rate",
    "Explanation": "Measures how often transactions are declined (due to insufficient credit or fraud checks). High decline rates suggest financial strain or credit overextension.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Significant MCC Changes",
    "Explanation": "Looks for notable shifts in merchant category codes (e.g., increased gambling). Sudden category changes can be an early warning of risky behavior.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Spending to Income Ratio",
    "Explanation": "Compares monthly credit usage to stated or estimated income. Excessive spending relative to income is a strong default predictor.",
    "Category": "Transaction Behavior"
  },
  {
    "Driver": "Card Tenure",
    "Explanation": "Gauges how long the borrower has held the credit card. Longer tenure often indicates loyalty and a more established repayment pattern.",
    "Category": "Account Tenure & Relationship"
  },
  {
    "Driver": "Issuer Card Count",
    "Explanation": "Counts how many active cards the borrower has with the same issuer. Multiple open cards may deepen loyalty but can also amplify credit exposure.",
    "Category": "Account Tenure & Relationship"
  },
  {
    "Driver": "Multi-Product Usage",
    "Explanation": "Examines whether the borrower also utilizes other banking products at the same institution. Responsible cross-product usage can lower overall risk.",
    "Category": "Account Tenure & Relationship"
  },
  {
    "Driver": "Account Opening Recency",
    "Explanation": "Focuses on how recently the account was opened. New accounts lack a payment track record, adding uncertainty to risk assessments.",
    "Category": "Account Tenure & Relationship"
  },
  {
    "Driver": "Account Tier Changes",
    "Explanation": "Captures upgrades or downgrades to different card types. Frequent tier changes can signal credit shopping or financial volatility.",
    "Category": "Account Tenure & Relationship"
  },
  {
    "Driver": "DPD Trend",
    "Explanation": "Monitors how Days Past Due progress over time. A rising trend signals increasing risk of default.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Delinquency Event Count",
    "Explanation": "Tallies how many times the borrower was significantly late (e.g., 30+ days). Multiple late episodes strongly correlate with default likelihood.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Delinquency Severity",
    "Explanation": "Differentiates short delinquencies (30 days) from severe ones (90+ days). Longer delinquencies indicate a higher probability of default.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Collection Agency Alert",
    "Explanation": "Flags if an account has been escalated to collections. This strongly predicts imminent default risk.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Grace Extension Frequency",
    "Explanation": "Measures how often the lender grants grace periods or re-ages the account. Repeated extensions imply persistent repayment difficulties.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Bounced Payment Count",
    "Explanation": "Tracks the number of payments returned for insufficient funds. Each bounced payment is a direct red flag for potential default.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Hard Inquiry Spike",
    "Explanation": "Checks how many recent hard inquiries appear on the borrower’s credit file. Sudden spikes may suggest credit shopping or financial distress.",
    "Category": "External Credit Behavior"
  },
  {
    "Driver": "External Collection Flags",
    "Explanation": "Indicates if the borrower has external accounts in collections or legal judgments. Suggests broader financial instability.",
    "Category": "External Credit Behavior"
  },
  {
    "Driver": "Bankruptcy or Lien Records",
    "Explanation": "Tracks whether the borrower has bankruptcies, liens, or similar public records. These strongly elevate default risk.",
    "Category": "External Credit Behavior"
  },
  {
    "Driver": "External Credit Usage Rate",
    "Explanation": "Measures the borrower’s credit utilization across all lenders. A high external usage rate heightens overall delinquency risk.",
    "Category": "External Credit Behavior"
  },
  {
    "Driver": "New Credit Lines Opened",
    "Explanation": "Counts how many new external credit accounts the borrower opened recently. Rapid credit expansion can foreshadow financial strain.",
    "Category": "External Credit Behavior"
  },
  {
    "Driver": "Payment to Income Ratio",
    "Explanation": "Compares monthly debt payments to monthly income. Higher ratios suggest limited capacity to handle new or rising obligations.",
    "Category": "Financial Capacity"
  },
  {
    "Driver": "Debt to Income Ratio",
    "Explanation": "Measures all debt obligations relative to income. A rising DTI typically indicates reduced flexibility and higher default probability.",
    "Category": "Financial Capacity"
  },
  {
    "Driver": "Disposable Income Estimate",
    "Explanation": "Approximates the leftover funds after essential expenses and debt. Lower disposable income correlates with higher risk of missed payments.",
    "Category": "Financial Capacity"
  },
  {
    "Driver": "Employment Continuity",
    "Explanation": "Evaluates the borrower’s job history stability. Frequent employment changes or job loss often increase default risk.",
    "Category": "Financial Capacity"
  },
  {
    "Driver": "Self-Requested Payment Deferrals",
    "Explanation": "Identifies if the borrower requested payment holidays or deferrals. Such requests often indicate short-term cashflow issues.",
    "Category": "Hardship & Forbearance"
  },
  {
    "Driver": "Lender-Driven Forbearance",
    "Explanation": "Notes if the institution granted forbearance or other relief measures. Multiple forbearances typically signal severe or ongoing hardship.",
    "Category": "Hardship & Forbearance"
  },
  {
    "Driver": "Borrower Contact Responsiveness",
    "Explanation": "Considers how promptly the borrower replies to communication attempts. Lack of response can precede default or abandonment of debt obligations.",
    "Category": "Customer Engagement"
  },
  {
    "Driver": "Customer Complaints Incidence",
    "Explanation": "Tracks the frequency of formal complaints or disputes raised by the borrower. High complaint volumes might indicate financial distress or dissatisfaction.",
    "Category": "Customer Engagement"
  },
  {
    "Driver": "Digital Login Activity",
    "Explanation": "Observes how regularly the borrower accesses their account online. A drop in logins may signal disengagement or looming repayment issues.",
    "Category": "Customer Engagement"
  },
  {
    "Driver": "Voluntary Limit Decrease",
    "Explanation": "Checks if the borrower lowers their own credit limit. This might reflect cautious self-management or signs of financial strain.",
    "Category": "Customer Engagement"
  },
  {
    "Driver": "Autopay Setting Changes",
    "Explanation": "Notes if the borrower disables or significantly modifies automatic payments. Cancelling autopay can be a precursor to missed payments.",
    "Category": "Customer Engagement"
  },
  {
    "Driver": "Income Verification Status",
    "Explanation": "Indicates if the borrower’s stated income was independently confirmed. Verified income typically reduces default risk compared to unverified amounts.",
    "Category": "Borrower Profile"
  },
  {
    "Driver": "Homeownership Flag",
    "Explanation": "Shows whether the borrower owns a home. Homeowners often have more stable equity and lower risk compared to renters.",
    "Category": "Borrower Profile"
  },
  {
    "Driver": "Public Records (10-Year)",
    "Explanation": "Counts bankruptcies, liens, or judgments in the last decade. Multiple records suggest prolonged financial difficulties.",
    "Category": "External Credit History"
  },
  {
    "Driver": "Loan Term Length",
    "Explanation": "Specifies the total repayment period (e.g., 36 or 60 months). Longer terms reduce monthly payments but extend default risk exposure.",
    "Category": "Loan Structure"
  },
  {
    "Driver": "Original Principal Amount",
    "Explanation": "The initial loan size. Larger loans can pose higher risk if the borrower’s capacity to repay is uncertain.",
    "Category": "Loan Structure"
  },
  {
    "Driver": "Job Tenure Duration",
    "Explanation": "How long the borrower has been in the same employment status. Extended job stability is typically associated with lower default rates.",
    "Category": "Borrower Profile"
  },
  {
    "Driver": "Active Revolving Accounts",
    "Explanation": "Counts the borrower’s currently open revolving lines (e.g., credit cards). More lines may increase capacity but can also signal higher debt exposure.",
    "Category": "Credit Behavior"
  },
  {
    "Driver": "Recent Trade Lines",
    "Explanation": "Number of newly opened credit trades (e.g., auto loan, credit card) within the last six months. Multiple new trades may indicate risky credit-seeking.",
    "Category": "Credit Behavior"
  },
  {
    "Driver": "Active Delinquencies",
    "Explanation": "Identifies obligations currently in delinquent status. Active delinquencies are a major sign of imminent default risk.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Annual Percentage Rate (APR)",
    "Explanation": "The loan’s effective interest cost to the borrower. Higher APR often signals higher perceived risk and increases payment burdens.",
    "Category": "Loan Pricing"
  },
  {
    "Driver": "Lifetime Trade Count",
    "Explanation": "All credit trades ever opened by the borrower. Although experience with credit can lower risk, excessive accounts may imply overextension.",
    "Category": "Credit History"
  },
  {
    "Driver": "Revolving Payment Total",
    "Explanation": "Sum of monthly payments across all revolving debts. High totals reduce leftover income for new or existing obligations.",
    "Category": "Payment Obligations"
  },
  {
    "Driver": "Delinquent Amount Owed",
    "Explanation": "Sum of all past-due balances across obligations. A substantial delinquent balance indicates acute financial trouble.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Bankcard Usage Rate",
    "Explanation": "Specifically measures credit card utilization relative to limits. High bankcard usage often increases default risk.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Active Credit Lines",
    "Explanation": "Number of total open credit lines, including both revolving and installment. Numerous active lines might indicate broad exposure to debt.",
    "Category": "Credit Behavior"
  },
  {
    "Driver": "Revolving Debt Balance",
    "Explanation": "Total outstanding sum on all revolving accounts. Large balances reduce borrowing power and can strain cash flow.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Income Bracket",
    "Explanation": "Groups the borrower’s earnings into low, medium, or high tiers. Lower brackets can correlate with tighter budgets and higher risk.",
    "Category": "Borrower Profile"
  },
  {
    "Driver": "Total Inquiry Count",
    "Explanation": "Reflects how many inquiries appear on the borrower’s credit file. Excessive inquiries often flag credit-seeking behavior.",
    "Category": "Credit Behavior"
  },
  {
    "Driver": "Available Credit on Bankcards",
    "Explanation": "Shows the remaining unused credit across all bankcards. Low available credit can lead to financial strain if unexpected expenses arise.",
    "Category": "Credit Utilization"
  },
  {
    "Driver": "Lender Profit Margin",
    "Explanation": "Represents the expected yield or return the lender aims to earn. Higher yields can mirror higher risk-based pricing and borrower vulnerability.",
    "Category": "Loan Pricing"
  },
  {
    "Driver": "Inquiries (Last 6 Months)",
    "Explanation": "Counts recent credit inquiries over the past half-year. A sudden uptick may signal distress or aggressive borrowing.",
    "Category": "Credit Behavior"
  },
  {
    "Driver": "Loan Purpose Category",
    "Explanation": "Specifies why the borrower took out the loan (e.g., refinancing, small business). Different loan purposes have varying risk profiles.",
    "Category": "Loan Structure"
  },
  {
    "Driver": "Current Days Overdue",
    "Explanation": "The exact number of days the loan is presently past due. Even a short delinquency can heighten default risk if it persists.",
    "Category": "Delinquency Indicators"
  },
  {
    "Driver": "Employment Category",
    "Explanation": "Shows if the borrower is full-time, part-time, self-employed, or unemployed. Employment stability underpins repayment capacity.",
    "Category": "Borrower Profile"
  },
  {
    "Driver": "Channel of Origination",
    "Explanation": "Indicates how the loan was originated (e.g., broker, retail). Certain channels historically correlate with higher or lower default rates.",
    "Category": "Origination Characteristics"
  },
  {
    "Driver": "Original Rate",
    "Explanation": "The interest rate at the time of loan signing. Higher original rates can strain monthly payments and elevate default potential.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Current Rate",
    "Explanation": "Any updated or adjusted interest rate. Rate hikes can stress the borrower’s budget, increasing risk of missed payments.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Outstanding Principal Balance",
    "Explanation": "The remaining loan amount after payments. Monitoring UPB trends helps gauge potential for default as the loan ages.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Loan Age in Months",
    "Explanation": "The number of months since origination. Early-stage loans often exhibit higher default risk, while well-aged loans may be safer.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Legal Maturity Countdown",
    "Explanation": "Months left until the final due date per the note. A shorter countdown sometimes reduces risk if enough principal has been repaid.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Property LTV at Origination",
    "Explanation": "Compares the mortgage amount to the property’s initial valuation. High LTV properties can leave borrowers with little equity buffer.",
    "Category": "Collateral & Equity"
  },
  {
    "Driver": "Combined LTV at Start",
    "Explanation": "Includes all debts secured by the home at origination. Higher combined leverage magnifies default risk in a downturn.",
    "Category": "Collateral & Equity"
  },
  {
    "Driver": "Co-Borrower FICO (Origination)",
    "Explanation": "Credit score of any co-applicant at the start. A low co-borrower score can pull overall creditworthiness down.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Refinance or Purchase Flag",
    "Explanation": "Indicates whether the loan was a purchase mortgage or a refinance. Cash-out refinances in particular may show higher risk behavior.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Property Use Case",
    "Explanation": "Defines the property as primary residence, secondary home, or investment. Non-owner occupancy usually has a higher default rate.",
    "Category": "Collateral & Equity"
  },
  {
    "Driver": "Geo-Region Code",
    "Explanation": "Broad geographic marker (e.g., MSA or partial ZIP code). Local economic trends, unemployment, and property values can influence default risk.",
    "Category": "Geographic & Market Factors"
  },
  {
    "Driver": "Mortgage Insurance Level",
    "Explanation": "The coverage percentage from mortgage insurance, if any. While MI can reduce lender losses, the borrower typically pays more, which can strain finances.",
    "Category": "Collateral & Insurance"
  },
  {
    "Driver": "ARM vs. Fixed Indicator",
    "Explanation": "States whether the loan has a variable or fixed interest rate. ARMs can introduce payment shocks, raising default likelihood.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Interest-Only Feature",
    "Explanation": "Notes if the loan starts with interest-only payments. Such structures can mask true affordability until principal repayments begin.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Delinquency Status",
    "Explanation": "Indicates how many months late the borrower is. Delinquency is the strongest near-term predictor of default or foreclosure.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Repayment History Snapshot",
    "Explanation": "Up to 24 months of pay history. Multiple or repeated delinquencies in that window strongly indicate default risk.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Modified Loan Flag",
    "Explanation": "Whether the loan terms have been altered due to hardship. A modification history can signal deeper financial trouble.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Mortgage Insurance Dropped",
    "Explanation": "If MI was removed after origination. Eliminating MI might lower payment but also removes insurance coverage, impacting net losses if default occurs.",
    "Category": "Collateral & Insurance"
  },
  {
    "Driver": "Zero Balance Reason",
    "Explanation": "Why the loan balance reached zero (e.g., full payoff, foreclosure). Different exit paths correlate with varied loss severities.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Date Loan Hit Zero",
    "Explanation": "When the balance officially reached zero. Timing helps determine prepayment speeds or the length of default resolution.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Principal at Removal",
    "Explanation": "The outstanding principal when a loan leaves the portfolio (e.g., via payoff or foreclosure). Important for final loss calculations.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Scheduled Principal Due",
    "Explanation": "The portion of each monthly payment meant to reduce principal. Tracking shortfalls can highlight emerging default trends.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Extra Principal Paid",
    "Explanation": "Any unscheduled principal payments. Borrowers who prepay principal may have lower risk, as they build equity faster.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Most Recent Paid Date",
    "Explanation": "The last installment date for which the lender received a complete payment. A gap since this date can signal looming delinquency.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Foreclosure Completion Date",
    "Explanation": "When legal foreclosure procedures concluded. Critical for modeling final recoveries and severities.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Property Liquidation Date",
    "Explanation": "Date when the lender or investor no longer retains an interest (e.g., sale of REO). This finalizes the credit event outcome.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Legal and Utility Costs",
    "Explanation": "Expenses tied to the foreclosure process, property valuation, and basic utilities. Higher costs inflate loss severity.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Maintenance and Repair Fees",
    "Explanation": "Expenditures to secure and preserve a foreclosed property. The longer the property remains unsold, the higher these costs can climb.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Eviction and Recovery Costs",
    "Explanation": "Funds spent removing occupants or personal property after default. Each additional cost magnifies overall losses.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "HOA and Other Holding Charges",
    "Explanation": "Expenses such as association dues or insurance premiums while holding a distressed property. These fees add to total foreclosure losses.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Property Tax Liabilities",
    "Explanation": "Taxes paid while the property is in foreclosure or REO. Unpaid taxes can create liens or hamper resale, raising severity.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Net Realization Proceeds",
    "Explanation": "The net cash recovered from selling the foreclosed property after fees. Drives final loss calculations on defaulted loans.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Insurance or Recourse Proceeds",
    "Explanation": "Any payouts from mortgage insurance or seller recourse. These proceeds offset losses sustained upon default.",
    "Category": "Collateral & Insurance"
  },
  {
    "Driver": "Repurchase Reimbursement",
    "Explanation": "Amounts collected if a lender repurchases a defective or ineligible loan. Helps mitigate losses in the securitized pool.",
    "Category": "Servicing & Administration"
  },
  {
    "Driver": "Additional Foreclosure Receipts",
    "Explanation": "Other post-foreclosure income (like redemption payments). They reduce overall loss on the defaulted asset.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Non-Interest-Bearing Portion",
    "Explanation": "Any principal set aside as non-interest-bearing in a modification. This typically indicates borrower hardship but can improve affordability.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Principal Write-Off",
    "Explanation": "An agreed-upon reduction in outstanding loan principal. While it eases borrower burden, it also signals severe financial stress.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Initial Listing Date",
    "Explanation": "When a distressed property first goes on the market (e.g., short sale). Time on market influences carrying costs and final loss.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Initial Asking Price",
    "Explanation": "The first listed price of the property. Overpricing can prolong market time; underpricing may expedite sale but reduce recovery.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Current Listing Date",
    "Explanation": "The most recent date the property was (re)listed. Frequent re-listings may indicate marketing challenges, increasing carrying costs.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Active Asking Price",
    "Explanation": "The latest listed sale price. Significant price drops can expedite liquidation but elevate potential losses.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Borrower FICO at Issuance",
    "Explanation": "Reflects the borrower’s credit score at the time the loan or pool was issued. Differences from original underwriting can signal changed risk.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Co-Borrower FICO at Issuance",
    "Explanation": "Co-borrower’s updated credit score at issuance. Divergence from the original might indicate shifting creditworthiness.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Borrower FICO Current",
    "Explanation": "Latest credit score for the primary borrower. Improvement may lower default risk, whereas declines can signal emerging problems.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Co-Borrower FICO Current",
    "Explanation": "Most recent credit score for any co-borrower. A drop in this score can undermine the overall repayment ability.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Mortgage Insurance Provider",
    "Explanation": "Who pays the MI premium (lender-paid vs. borrower-paid). While it doesn’t directly lower borrower risk, it can shift cost burdens.",
    "Category": "Collateral & Insurance"
  },
  {
    "Driver": "Servicing Transfer Flag",
    "Explanation": "Indicates if the servicing rights changed hands recently. Servicing transfers can cause payment disruptions or confuse borrowers.",
    "Category": "Servicing & Administration"
  },
  {
    "Driver": "Most Recent Modification Loss",
    "Explanation": "Loss realized in the latest reporting period due to a loan modification. Reflects ongoing financial distress for the borrower.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Cumulative Modification Losses",
    "Explanation": "Totals all losses from modifications over the life of the loan. Persistent or repeated modifications signal high borrower risk.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Current Period Credit Loss/Gain",
    "Explanation": "Shows realized gain or loss from a credit event (e.g., foreclosure) for the current period. Large losses indicate significant default impact.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Total Credit Loss/Gain to Date",
    "Explanation": "Aggregates all gains or losses from credit events since the loan’s inception. Sizable negative totals highlight repeated or severe defaults.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Special Program Indicator",
    "Explanation": "Flags affordable or refinance programs (e.g., HomeReady). Program-specific underwriting can affect risk patterns.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Foreclosure Statute Write-Off",
    "Explanation": "Principal deemed uncollectable under state foreclosure laws. Directly increases net loss after a default event.",
    "Category": "Foreclosure & Disposal"
  },
  {
    "Driver": "Relocation Loan Flag",
    "Explanation": "Shows if the mortgage was provided for a job relocation. Employer assistance can mitigate risk, but job volatility can also increase it.",
    "Category": "Borrower & Credit Profile"
  },
  {
    "Driver": "Zero Balance Status Change Date",
    "Explanation": "The latest date on which a zero-balance code (payoff, foreclosure) was updated. Helps trace the loan’s final outcome timeline.",
    "Category": "Loan Performance Tracking"
  },
  {
    "Driver": "Loan Holdback Status",
    "Explanation": "Indicates if the loan is held in a temporary status for review (e.g., data discrepancies, special forbearance). Such loans can carry uncertain risk.",
    "Category": "Servicing & Administration"
  },
  {
    "Driver": "Holdback Effective Date",
    "Explanation": "When the holdback status was most recently applied or changed. Pinpoints when the loan deviated from normal servicing.",
    "Category": "Servicing & Administration"
  },
  {
    "Driver": "Accrued Interest in Delinquency",
    "Explanation": "Unpaid interest piling up on a delinquent loan. Elevated delinquent interest typically signals deeper financial distress.",
    "Category": "Payment & Performance"
  },
  {
    "Driver": "Valuation Method Used",
    "Explanation": "Specifies how the property’s value was determined (e.g., appraisal, AVM). Inaccurate valuation at origination can increase default risk if LTV is misjudged.",
    "Category": "Collateral & Equity"
  },
  {
    "Driver": "High-Balance Mortgage Flag",
    "Explanation": "Indicates the original loan exceeded standard conforming limits but remained within high-cost limits. Such loans can show distinct risk and prepayment patterns.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Short ARM Fixed Period",
    "Explanation": "Whether the initial fixed-rate period on an ARM is five years or less. Shorter fixed windows can lead to quicker payment shocks.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "ARM Type Description",
    "Explanation": "Outlines the structure of an adjustable-rate mortgage (initial fixed term, adjustment frequency). Certain ARM types are more prone to volatility.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Months to Initial Rate Reset",
    "Explanation": "How long until the first ARM rate adjustment. The sooner the reset, the earlier the borrower faces potential payment hikes.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Rate Adjustment Interval",
    "Explanation": "The frequency of subsequent interest rate adjustments on an ARM. More frequent resets can magnify payment unpredictability.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Next ARM Rate Change",
    "Explanation": "Date when the ARM’s interest rate will adjust. A looming rate increase may trigger default if the borrower’s budget is tight.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Next Payment Adjustment Date",
    "Explanation": "When the borrower’s monthly payment will change (potentially distinct from the rate change date). Misalignment can catch borrowers off-guard.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "ARM Reference Index",
    "Explanation": "The underlying index (e.g., LIBOR, SOFR) used to adjust ARM rates. Certain indices exhibit higher volatility, affecting payment risk.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "ARM Cap Limits",
    "Explanation": "Summarizes the initial, periodic, and lifetime cap on interest rate increases. Higher caps can result in larger payment jumps.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Initial Rate Jump Cap",
    "Explanation": "The maximum percentage the interest rate can rise at the first adjustment. A large jump can induce significant payment shock.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Subsequent Rate Jump Cap",
    "Explanation": "Maximum upward adjustment for each future reset. Higher periodic caps amplify the risk of a payment spike mid-loan.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Lifetime Rate Ceiling",
    "Explanation": "Absolute limit on how high the rate can go over the life of the loan. A wide cap range exposes the borrower to more risk.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Interest Margin",
    "Explanation": "The constant spread added to the index to form the total ARM rate. A higher margin typically indicates steeper payments.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Balloon Payment Feature",
    "Explanation": "Whether the loan includes a large, one-time principal payment at maturity. Balloon loans can create default spikes when the payment is due.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "ARM Plan Code",
    "Explanation": "A standardized reference for the specific ARM design. Certain plan codes may have historically higher or lower default incidence.",
    "Category": "Loan Terms"
  },
  {
    "Driver": "Hardship Assistance Program",
    "Explanation": "Shows if the borrower is in a temporary relief or repayment plan. Indicative of past or ongoing financial strain.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "HLTV Refinance Tag",
    "Explanation": "Flags a high-LTV refinance scenario where the loan remains in the reference pool. High LTV can exacerbate default risk under price declines.",
    "Category": "Collateral & Equity"
  },
  {
    "Driver": "Repurchase Make-Whole Flag",
    "Explanation": "Indicates if rep/warranty settlement proceeds were received. While it offsets investor losses, such loans are often from riskier cohorts.",
    "Category": "Servicing & Administration"
  },
  {
    "Driver": "Alternative Delinquency Cure",
    "Explanation": "Whether the borrower is enrolled in a forbearance, repayment plan, or trial mod. Points to active delinquency management.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Alternative Resolution Count",
    "Explanation": "Number of times the borrower has attempted a delinquency cure plan. Multiple attempts can signal chronic repayment troubles.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Cumulative Deferral Amount",
    "Explanation": "Total unpaid principal set aside for later. Large deferrals might reduce immediate risk but concentrate repayment burdens in the future.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Deferral-Driven Modification Flag",
    "Explanation": "Shows if a payment deferral triggered a modification event. Indicates deeper or repeated borrower hardship.",
    "Category": "Servicing & Modification"
  },
  {
    "Driver": "Interest-Bearing UPB",
    "Explanation": "Current principal that still accrues interest after any modifications. A smaller interest-bearing balance can ease monthly payments, lowering short-term risk.",
    "Category": "Loan Performance Tracking"
  }
]

# Convert to pandas DataFrame
df = pd.DataFrame(long_list_dict)

df.sample(5)

,Driver,Explanation,Category
92,Mortgage Insurance Dropped,"If MI was removed after origination. Eliminating MI might lower payment but also removes insurance coverage, impacting net losses if default occurs.",Collateral & Insurance
136,ARM Type Description,"Outlines the structure of an adjustable-rate mortgage (initial fixed term, adjustment frequency). Certain ARM types are more prone to volatility.",Loan Terms
56,Active Revolving Accounts,"Counts the borrower’s currently open revolving lines (e.g., credit cards). More lines may increase capacity but can also signal higher debt exposure.",Credit Behavior
106,Net Realization Proceeds,The net cash recovered from selling the foreclosed property after fees. Drives final loss calculations on defaulted loans.,Foreclosure & Disposal
50,Income Verification Status,Indicates if the borrower’s stated income was independently confirmed. Verified income typically reduces default risk compared to unverified amounts.,Borrower Profile


### D. High-level stats

In [4]:
# Stats
df.groupby('Category')['Driver'].count().reset_index().sort_values('Driver', ascending=False)

,Category,Driver
19,Loan Terms,22
13,Foreclosure & Disposal,16
7,Credit Utilization,11
25,Servicing & Modification,10
9,Delinquency Indicators,9
22,Payment Behavior,8
16,Loan Performance Tracking,7
26,Transaction Behavior,7
1,Borrower & Credit Profile,6
21,Payment & Performance,6
